In [1]:
pip install lingualytics

  Using cached lingualytics-0.1.3-py3-none-any.whl (12 kB)
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Using cached texthero-1.1.0-py3-none-any.whl (24 kB)
  Using cached plotly-5.3.1-py2.py3-none-any.whl (23.9 MB)
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
Note: you may need to restart the kernel to use updated packages.


VIRTUAL ASSISTANT 

In [2]:
import numpy as np

In [9]:
from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import hi_stopwords,en_stopwords
from texthero.preprocessing import remove_digits
import pandas as pd

In [44]:
Dataset=r"D:\faqs.csv"

In [49]:
df = pd.read_csv(Dataset,encoding='cp1252')
df

,Q,A
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [125]:
# df = pd.read_csv(Dataset,encoding='cp1252')
# pd.set_option('display.max_colwidth', None)
df['clean_Q'] = df['Q'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) \
#                                     .pipe(remove_lessthan,length=3) \
#                                     .pipe(remove_stopwords,stopwords=en_stopwords.union(hi_stopwords))
df

,Q,A,clean_Q
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [55]:
pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in c:\users\admin\anaconda3\lib\site-packages (2.1.0)



In [63]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [66]:
q_embs = model.encode(df["clean_Q"], convert_to_tensor=True)

In [192]:
q_embs.shape

torch.Size([1, 384])

In [193]:
user_query="kandi means"

In [194]:
df_user_q=pd.DataFrame([user_query],columns=["user_query"])
df_user_q

,user_query
0,kandi means


In [195]:
df_user_q['clean_user_Q'] = df_user_q['user_query'].pipe(remove_digits)\
                                    .pipe(remove_punctuation) 
df_user_q


,user_query,clean_user_Q
0,kandi means,kandi means


In [204]:
uq_embs = model.encode(df_user_q["clean_user_Q"],convert_to_tensor=True)
uq_embs.shape

torch.Size([1, 384])

In [197]:
import torch

In [198]:
cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos_fn

CosineSimilarity()

In [199]:
# cos_fn(uq_embs,q_embs)
idx=np.argmax(cos_fn(uq_embs,q_embs)).item()

In [200]:
df["A"][idx]

'kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [201]:
def pred_ans(userr_query):
    df_user_q=pd.DataFrame([userr_query],columns=["userr_query"])
    df_user_q['clean_user_Q'] = df_user_q['userr_query'].pipe(remove_digits)\
                            .pipe(remove_punctuation) 
    uq_embs = model.encode(df_user_q["clean_user_Q"],convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
    idx=np.argmax(cos_fn(uq_embs,q_embs)).item()
    return df['A'][idx]

In [202]:
while True:
    userr_query=input("Ask a question: ")
    if userr_query == "exit":
        break
    else:
        print(pred_ans(userr_query))
        print("---------------------")

Ask a question: what does kandi meant to be
kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.
---------------------
Ask a question: exit
